In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [ ]:
(train_ds, train_labels), (test_ds, test_labels) = tfds.load("tf_flowers", split=["train[:70%]", "train[:30%]"],
                                                             batch_size=-1, as_supervised=True)

In [ ]:
print(train_ds.shape)
print(train_ds[0].shape)

In [ ]:
#resizing images
train_ds = tf.image.resize(train_ds, (150,150))
test_ds = tf.image.resize(test_ds, (150,150))

In [ ]:
train_labels

In [ ]:
#Transform labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [ ]:
train_labels[0]

In [ ]:
#use pretrained VGG16 image Classification model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
#loading VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=train_ds[0].shape)

In [ ]:
# will not train base model
base_model.trainable = False
#preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

In [ ]:
# model details
base_model.summary()

In [ ]:
#add our layers on top of the model
from tensorflow.keras import layers, models
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

ml= models.Sequential([base_model, flatten_layer, dense_layer_1, dense_layer_2, prediction_layer])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
ml.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

In [ ]:
ml.fit(train_ds, train_labels, epochs=10, validation_split=0.2, batch_size=32, callbacks=[es])

In [ ]:
ml.evaluate(test_ds, test_labels)

In [ ]:
#model build from scratch
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# layers
ml = Sequential()
ml.add(Rescaling(1./255, input_shape=(150,150,3)))
ml.add(layers.Conv2D(16,kernel_size=8, activation='relu'))
ml.add(layers.MaxPooling2D(3))
ml.add(layers.Conv2D(32,kernel_size=8, activation='relu'))
ml.add(layers.MaxPooling2D(2))
ml.add(layers.Conv2D(32,kernel_size=8, activation='relu'))
ml.add(layers.MaxPooling2D(2))

# Flatten
ml.add(layers.Flatten())
ml.add(layers.Dense(50, activation='relu'))
ml.add(layers.Dense(20, activation='relu'))
ml.add(layers.Dense(5, activation='softmax'))

# Compile
ml.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'],)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)
ml.fit(train_ds,train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

In [ ]:
ml.evaluate(test_ds, test_labels)